In [1]:
import pandas as pd
import os

In [2]:
Path = "C:/Users/doosti/OneDrive - Chapman University/Research/Research Projects/Fitness/Data/"
print(Path)

C:/Users/doosti/OneDrive - Chapman University/Research/Research Projects/Fitness/Data/


In [3]:
with open(Path + "video_info_pilates.csv", "r", encoding='utf-8') as f:
    lines = f.readlines()
    lines = lines[1:]
    lines = [line.split(",") for line in lines]

for i,line in enumerate(lines):
    if len(line) > 3:
        lines[i] = [line[0], line[1], ",".join(line[2:])]

videos = pd.DataFrame(lines, columns=["date", "creator_id", "title"])
print(videos.shape)
print(videos.head())
print(videos.isnull().sum())

(116023, 3)
                   date                creator_id  \
0  2024-03-31T18:41:56Z  UCzzYnZ8GIzfB1Vr3hk2Nj9Q   
1  2024-03-31T18:12:27Z  UCzzYnZ8GIzfB1Vr3hk2Nj9Q   
2  2024-03-30T19:58:47Z  UCzzYnZ8GIzfB1Vr3hk2Nj9Q   
3  2024-03-30T18:31:11Z  UCzzYnZ8GIzfB1Vr3hk2Nj9Q   
4  2024-03-30T00:30:27Z  UCzzYnZ8GIzfB1Vr3hk2Nj9Q   

                                               title  
0  MTS Nutrition Tyrant Estogen and Cortisol Cont...  
1          This Moron Brings Coconuts on Airplanes\n  
2                         The most natty bar ever!\n  
3  MTS Nutrition Zinc + Magnesium + B6 Review | T...  
4                              I'm Leaving YouTube\n  
date           0
creator_id    11
title         11
dtype: int64


In [4]:
# just keep the date and remove the time
videos['date'] = videos['date'].str.split("T").str[0]
videos['date'] = pd.to_datetime(videos['date'], errors='coerce')
videos = videos[videos.date.notnull()]
videos.head()

,date,creator_id,title
0,2024-03-31,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,MTS Nutrition Tyrant Estogen and Cortisol Cont...
1,2024-03-31,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,This Moron Brings Coconuts on Airplanes\n
2,2024-03-30,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,The most natty bar ever!\n
3,2024-03-30,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,MTS Nutrition Zinc + Magnesium + B6 Review | T...
4,2024-03-30,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,I'm Leaving YouTube\n


In [5]:
import re
def flag_challenges(text):
    """
    flags a string to determine they include keywords "challenge" or "Day #"
    text (str): video title
    return (bool) if flagged
    """
    return (len(re.search('day \d+|$', text.lower()).group()) > 0 ) | ('challenge' in text.lower())

def find_challenge_videos(video_titles):
    """
    flags videos that have challenge or day in their title
    videos (pandas.Series): a pandas series of video titles to be flagged
        each video is a dictionary
    return (pandas.Series)
    """
    return video_titles.apply(flag_challenges)


In [6]:
videos['challenge'] = find_challenge_videos(videos.title)

In [7]:
videos.challenge.value_counts()

False    111845
True       4167
Name: challenge, dtype: int64

In [8]:
trends = pd.read_csv(Path + "trends_Physical Exercise_Physical Fitness_Gymnasium_Wellness_Strength Training_Pilates.csv")
trends.head()

,creator_id,date,views,engagements,uploads,followers
0,2x5Qizg7id,2020-08-08,13128,230,1,NaN
1,2x5Qizg7id,2020-08-09,12953,201,0,NaN
2,2x5Qizg7id,2020-08-10,12290,182,0,NaN
3,2x5Qizg7id,2020-08-11,15276,418,2,NaN
4,2x5Qizg7id,2020-08-12,16044,375,1,NaN


In [10]:
creators = pd.read_csv(Path + "creators_Physical Exercise_Physical Fitness_Gymnasium_Wellness_Strength Training_Pilates.csv")
creators.head()

,creator_id,title,type,youtube_id,country,language,genre,industry,uploads_90d,views,last_upload_date
0,Dh9aKQRw503PUa1V01,Centrum PH,brand,yta_DmSmAwt4X9gMudgRwJ3ecQ,PH,en,"Health, Fitness & Self Help",Personal Care & Health,2,462701069,2023-07-18
1,Cp3ULAtU6I,Fit Tuber,brand,yta_YC6Vcczj8v-Y5OgpEJTFBw,IN,en,"Health, Fitness & Self Help",Personal Care & Health,6,852196427,2023-07-28
2,212xVuk3Jf03RKHn201,Just Move Zumba Dance Fitness Milan Italy,brand,yta_Zo8BAi0leaHpPgY1kAmwzA,PH,en,"Health, Fitness & Self Help",Personal Care & Health,101,244559967,2023-07-29
3,GcEzlKE2sH,Bob & Brad,brand,yta_mTe0LsfEbpkDpgrxKAWbRA,US,en,"Health, Fitness & Self Help",Personal Care & Health,417,619341858,2023-08-06
4,KFN1qx7ryz,Sethu,brand,yta_0KMy2IyOb8BHz05Kxz-YCQ,IN,en,"Health, Fitness & Self Help",Personal Care & Health,97,164419334,2023-08-06


In [11]:
# removing 'yta' from youtube_id
creators['youtube_id'] = creators['youtube_id'].str.replace('yta_', '')
# extract creator_id and youtube_id into a dictionary (keys: creator_id, values: youtube_id)
creator_dict = creators.set_index('youtube_id')['creator_id'].to_dict()
# remove the first two letters from creator_id
videos['creator_id'] = videos.creator_id.str[2:]

In [12]:
videos[videos.creator_id.isin(list(creator_dict.keys()))].shape

(116012, 4)

In [13]:
videos.creator_id

0         zzYnZ8GIzfB1Vr3hk2Nj9Q
1         zzYnZ8GIzfB1Vr3hk2Nj9Q
2         zzYnZ8GIzfB1Vr3hk2Nj9Q
3         zzYnZ8GIzfB1Vr3hk2Nj9Q
4         zzYnZ8GIzfB1Vr3hk2Nj9Q
                   ...          
116018    sedpgo_lnZ6SQmn__uH8ZA
116019    sedpgo_lnZ6SQmn__uH8ZA
116020    sedpgo_lnZ6SQmn__uH8ZA
116021    sedpgo_lnZ6SQmn__uH8ZA
116022    sedpgo_lnZ6SQmn__uH8ZA
Name: creator_id, Length: 116012, dtype: object

In [14]:
# get alternative id by using a dictionary and replace creator_id
videos['creator_id'] = videos.creator_id.map(creator_dict)

In [15]:
videos[['creator_id','date']]

,creator_id,date
0,2x5Qizg7id,2024-03-31
1,2x5Qizg7id,2024-03-31
2,2x5Qizg7id,2024-03-30
3,2x5Qizg7id,2024-03-30
4,2x5Qizg7id,2024-03-30
...,...,...
116018,IAn1PTOoUh,2018-01-19
116019,IAn1PTOoUh,2018-01-17
116020,IAn1PTOoUh,2018-01-11
116021,IAn1PTOoUh,2018-01-05


In [19]:
videos.groupby(['creator_id','date']).size().reset_index().rename(columns={0:'count'})

,creator_id,date,count
0,0ZLkVcZ76g,2019-09-30,1
1,0ZLkVcZ76g,2020-02-03,1
2,0ZLkVcZ76g,2020-02-06,1
3,0ZLkVcZ76g,2020-02-12,1
4,0ZLkVcZ76g,2020-02-14,1
...,...,...,...
72553,zIDBEkDAoy,2021-07-27,7
72554,zIDBEkDAoy,2021-10-05,1
72555,zIDBEkDAoy,2022-01-09,1
72556,zIDBEkDAoy,2022-12-01,1


In [20]:
trends['date'] = pd.to_datetime(trends['date'], errors='coerce')
grouped = videos.groupby(['creator_id','date']).challenge.sum().reset_index()
total_videos = videos.groupby(['creator_id','date']).size().reset_index().rename(columns={0:'count'})
grouped['non_challenge'] = total_videos['count'] - grouped['challenge']
trends['challenge'] = trends.merge(grouped, on=['creator_id','date'], how='left').challenge.fillna(0)
trends['non_challenge'] = trends.merge(grouped, on=['creator_id','date'], how='left').non_challenge.fillna(0)
print(trends.head())

   creator_id       date  views  engagements  uploads  followers  challenge  \
0  2x5Qizg7id 2020-08-08  13128          230        1        NaN        0.0   
1  2x5Qizg7id 2020-08-09  12953          201        0        NaN        0.0   
2  2x5Qizg7id 2020-08-10  12290          182        0        NaN        0.0   
3  2x5Qizg7id 2020-08-11  15276          418        2        NaN        0.0   
4  2x5Qizg7id 2020-08-12  16044          375        1        NaN        0.0   

   non_challenge  
0            1.0  
1            0.0  
2            0.0  
3            2.0  
4            1.0  


In [21]:
trends['challenge_dummy'] = trends['challenge'] > 0
trends['challenge_dummy'] = trends['challenge_dummy'].astype(int)
print(trends.value_counts('challenge_dummy'))

trends['non_challenge_dummy'] = trends['non_challenge'] > 0
trends['non_challenge_dummy'] = trends['non_challenge_dummy'].astype(int)
print(trends.value_counts('non_challenge_dummy'))

challenge_dummy
0    217829
1      1171
dtype: int64
non_challenge_dummy
0    190941
1     28059
dtype: int64


In [22]:
trends.to_csv(Path + "trends_nonYoga.csv", index=False)

In [93]:
# find channels that have no challenge videos
trends.groupby('creator_id').challenge_dummy.sum().value_counts()

0      103
1       31
2       19
3        9
6        6
9        4
5        2
8        2
15       2
76       1
26       1
67       1
66       1
64       1
56       1
82       1
49       1
31       1
28       1
18       1
22       1
21       1
20       1
19       1
14       1
13       1
12       1
11       1
7        1
4        1
241      1
Name: challenge_dummy, dtype: int64